In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import sys


project_root = Path.cwd()
while not (project_root / "src").exists() and project_root != project_root.parent:
    project_root = project_root.parent

src_path = project_root / "src"
fig_dir = project_root / "figures"
res_dir = project_root / "results"

fig_dir.mkdir(exist_ok=True)
res_dir.mkdir(exist_ok=True)

print("CWD:", Path.cwd())
print("project_root:", project_root)
print("src_path:", src_path, "exists:", src_path.exists())

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from model_proteostasis import ModelParams, step_mutations



CWD: C:\Users\Regi\Documents\Replication-Asymmetry-Model\notebooks
project_root: C:\Users\Regi\Documents\Replication-Asymmetry-Model
src_path: C:\Users\Regi\Documents\Replication-Asymmetry-Model\src exists: True


In [6]:
p = ModelParams()

rng = np.random.default_rng(42)
size = 100_000

M0 = step_mutations(rng, asym=False, size=size, p=p)
M1 = step_mutations(rng, asym=True,  size=size, p=p)

mean0 = M0.mean()
mean1 = M1.mean()
ratio = mean1 / mean0

print("E[M0] ≈", mean0)
print("E[M1] ≈", mean1)
print("ratio ≈", ratio)

df_mut = pd.DataFrame({"M0": M0, "M1": M1})
df_mut.describe()


E[M0] ≈ 0.03609
E[M1] ≈ 0.10107
ratio ≈ 2.800498753117207


,M0,M1
count,100000.000000,100000.000000
mean,0.036090,0.101070
std,0.190073,0.318427
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,3.000000,3.000000


In [7]:
stats = {
    "mean_M0": float(mean0),
    "mean_M1": float(mean1),
    "ratio_M1_over_M0": float(ratio),
    "size": int(size),
}

df_stats = pd.DataFrame([stats])
out_path = res_dir / "mutation_sanity_stats.csv"
df_stats.to_csv(out_path, index=False)
print("Saved:", out_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\results\mutation_sanity_stats.csv


In [8]:
plt.figure()
bins = np.arange(0, M1.max() + 2) - 0.5

plt.hist(M0, bins=bins, alpha=0.5, density=True, label="H0 (symmetric)")
plt.hist(M1, bins=bins, alpha=0.5, density=True, label="H1 (asymmetric)")

plt.xlabel("Mutation count per division")
plt.ylabel("Density")
plt.title("Mutation count distribution (H0 vs H1)")
plt.legend()

fig_path = fig_dir / "mutation_sanity_histogram.png"
plt.savefig(fig_path, dpi=300, bbox_inches="tight")
plt.close()
print("Saved:", fig_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\figures\mutation_sanity_histogram.png
